Будем использовать физическими величинами с измерениями. Модуль PhysicalQuantities сломался, не обновлялся два года, поэтому придется попробовать новое.

In [8]:
from pint import UnitRegistry
from uncertainties import ufloat

In [9]:
ureg = UnitRegistry()
length = ufloat(10, 0.5) * ureg.meter
print(length)  # (10 ± 0.5) m


10.0+/-0.5 meter


In [12]:
H_отсчета = ufloat(200, 10) * ureg.kilometer
H_отсчета

<Quantity(200.000+/-10.000, 'kilometer')>

In [10]:
P_отсчета =  ufloat(8.53, 1)* 1e-5 * ureg.pascal
P_отсчета

<Quantity((8.53000+/-1.00000)e-05, 'pascal')>

In [13]:
H = ufloat(60, 10) * ureg.kilometer
H

<Quantity(60.0000+/-10.0000, 'kilometer')>